In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

dates = pd.date_range(start="1995-01-01", end="2024-12-01", freq="ME")
n = len(dates)

# Macro variables (random walks)
interest_rate = np.cumsum(np.random.normal(0, 0.02, n)) + 5
inflation = np.cumsum(np.random.normal(0, 0.01, n)) + 2
gdp_growth = np.random.normal(2, 0.5, n)
unemployment = np.cumsum(np.random.normal(0, 0.02, n)) + 6
population_growth = np.random.normal(0.2, 0.05, n)
housing_starts = np.random.normal(100000, 5000, n)
rent_index = np.cumsum(np.random.normal(0.1, 0.05, n)) + 100

# Housing price index (depends on macro + trend)
trend = np.linspace(100, 350, n)
noise = np.random.normal(0, 5, n)
price_index = trend + 10*inflation - 5*interest_rate + noise

df = pd.DataFrame({
    "date": dates,
    "price_index": price_index,
    "interest_rate": interest_rate,
    "inflation": inflation,
    "gdp_growth": gdp_growth,
    "unemployment": unemployment,
    "population_growth": population_growth,
    "housing_starts": housing_starts,
    "rent_index": rent_index
})

df.to_csv("synthetic_housing_data.csv", index=False)
df.head()


In [ ]:
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("synthetic_housing_data.csv", parse_dates=["date"])
df.set_index("date", inplace=True)

X = df.drop(columns=["price_index"])
y = np.log(df["price_index"])
y_mean = y.mean()
y_std = y.std()
y_scaled = (y - y_mean) / y_std

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def create_sequences(X, y, window=60):
    Xs, ys = [], []
    for i in range(len(X) - window):
        Xs.append(X[i:i+window])
        ys.append(y[i+window])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y.values, window=60)


In [ ]:
print(X_seq.shape)
print(y_seq.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

model = Sequential([
    Input(shape=(X_seq.shape[1], X_seq.shape[2])),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

In [ ]:
model.fit(X_seq, y_seq, epochs=10, batch_size=32)

In [ ]:
def simulate_macro(last_row, years=30, steps_per_year=12):
    steps = years * steps_per_year
    current = last_row.copy()
    sims = []
    
    for _ in range(steps):
        noise = np.random.normal(0, 0.01, len(current))
        current = current * (1 + noise)
        sims.append(current.copy())
        
    return np.array(sims)

future_macro = simulate_macro(X_scaled[-1])


In [ ]:
# model outputs scaled values
predictions_scaled = []

for step in future_macro:
    window = np.vstack([window[1:], step])
    pred = model.predict(window.reshape(1, *window.shape), verbose=0)
    predictions_scaled.append(pred[0,0])

# ONLY here:
predictions = np.exp(np.array(predictions_scaled) * y_std + y_mean)


In [ ]:
print(df["price_index"].head())
print(y.head())

In [ ]:
import matplotlib.pyplot as plt

future_dates = pd.date_range(start=df.index[-1], periods=len(predictions), freq="ME")

plt.figure(figsize=(12,5))
plt.plot(df.index, df["price_index"], label="Historical")
plt.plot(future_dates, predictions, label="30-Year Forecast")
plt.legend()
plt.title("Housing Price Forecast (Hackathon Demo)")
plt.show()